# Install Dependencies

## External Dependencies

In [1]:
!pip install inflect


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


## FrostAura Dependencies

In [2]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

## Logging

In [3]:
import logging

logging.getLogger().disabled = True

## Imports

In [4]:
from frostaura.engines import IAssetProjectionEngine
from frostaura.engines import SimpleAssetProjectionEngine
import pandas as pd
from frostaura.engines import IAssetValuationEngine
from frostaura.engines import FinvizAssetValuationEngine
from frostaura.engines import IAssetCalculationsEngine
from frostaura.engines import SimpleAssetCalculationsEngine
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import HtmlResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import IPublicAssetDataAccess
from frostaura.data_access import YahooFinanceDataAccess
from frostaura.models import ValuationResult

# Usage

## Setup

In [5]:
html_data_access: IResourcesDataAccess = HtmlResourcesDataAccess()
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
asset_valuation_engine: IAssetValuationEngine = FinvizAssetValuationEngine(html_data_access=html_data_access)
public_asset_data_access: IPublicAssetDataAccess = YahooFinanceDataAccess()
asset_calculations_engine: IAssetCalculationsEngine = SimpleAssetCalculationsEngine(public_asset_data_access=public_asset_data_access)
personal_asset_data_access: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username=None,
                                                                         password=None)
instance: IAssetProjectionEngine = SimpleAssetProjectionEngine(asset_valuation_engine=asset_valuation_engine,
                                                               public_asset_data_access=public_asset_data_access)

## Project for a Single Symbol

In [6]:
n_months: int = 12*30 # 39 year
usd_zar_exchange_rate: float = 16.17
symbol: str = 'AAPL'
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
initial_value: float = holdings_with_profits.loc[holdings_with_profits['symbol'] == 'AAPL']['total_current_usd'].sum()
monthly_deposit: float = 2000 / usd_zar_exchange_rate / len(holdings_with_profits)

growth: pd.DataFrame = instance.project_monthly_asset_growth(n_months=n_months,
                                                             symbol=symbol,
                                                             principal_value=initial_value,
                                                             monthly_deposit=monthly_deposit)

growth

,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
month,,,,,
0,0.000000,0.000000,0.000000,0.000000,74.697050
1,30.921459,0.611893,30.921459,0.611893,106.230403
2,30.921459,0.870204,61.842919,1.482097,138.022067
3,30.921459,1.130631,92.764378,2.612728,170.074157
4,30.921459,1.393191,123.685838,4.005919,202.388807
...,...,...,...,...,...
356,30.921459,539.994072,11008.039579,55408.106306,66490.842936
357,30.921459,544.670822,11038.961039,55952.777128,67066.435218
358,30.921459,549.385882,11069.882498,56502.163010,67646.742559


## Project for a Entire Holdings (Multiple Symbols)

In [7]:
# 30 years
n_months: int = 12*30
usd_zar_exchange_rate: float = 16.17
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
# Simply R 1000 converted to USD and split evenly among how many assets are held.
monthly_deposits: list = [(1000 / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
holdings_growth = instance.project_monthly_holdings_growth(n_months=n_months,
                                                           holdings_with_profits=holdings_with_profits,
                                                           monthly_deposits=monthly_deposits)

holdings_growth

,month,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
0,1,61.842919,0.023089,61.842919,6.103659e+00,3.261989e+02
1,2,61.842919,0.022559,123.685838,1.363233e+01,3.955705e+02
2,3,61.842919,0.022247,185.528757,2.263265e+01,4.664137e+02
3,4,61.842919,0.022058,247.371676,3.315302e+01,5.387770e+02
4,5,61.842919,0.021948,309.214595,4.524369e+01,6.127106e+02
...,...,...,...,...,...,...
355,356,61.842919,0.041393,22016.079159,1.480936e+09,1.480958e+09
356,357,61.842919,0.041394,22077.922078,1.544886e+09,1.544909e+09
357,358,61.842919,0.041395,22139.764997,1.611600e+09,1.611622e+09
358,359,61.842919,0.041396,22201.607916,1.681196e+09,1.681218e+09


## Project Wealth Over Months

In [8]:
monthly_zar_deposit: float = 2000
usd_zar_exchange_rate: float = 16.17
monthly_deposits: list = [(monthly_zar_deposit / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
wealth_projection = instance.project_wealth_growth(holdings_with_profits=holdings_with_profits,
                                                   monthly_deposits=monthly_deposits)

wealth_projection

,Target Balance (USD),Target Balance Eng (USD),ETA (Months),ETA (Years)
0,1078.65,one thousand and seventy-eight point six four ...,5,0.416667
1,10263.25,"ten thousand, two hundred and sixty-three poin...",43,3.583333
2,102258.12,"one hundred and two thousand, two hundred and ...",107,8.916667
3,1014459.67,"one million, fourteen thousand, four hundred a...",166,13.833333
4,10154962.33,"ten million, one hundred and fifty-four thousa...",222,18.500000
5,101839639.44,"one hundred and one million, eight hundred and...",277,23.083333
6,1036323259.17,"one billion, thirty-six million, three hundred...",332,27.666667
7,10161642528.27,"ten billion, one hundred and sixty-one million...",386,32.166667
